In [1]:
%config IPCompleter.greedy=True

In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, Module

ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')

In [2]:
# Load modules from workspace
train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Train')
score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')

# Load a module from local as anonymous module
#a_module_in_development_func = Module.load_from_yaml(ws, path='/home/code/new_module/spec.yaml')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [3]:
help(train_module_func)

Help on function microsoft.com/aml/samples://Train in microsoft:

microsoft.com/aml/samples://Train(training_data: 'Training data organized in the torchvision format/structure' = None, max_epochs: 'Maximum number of epochs for the training, default is 5' = '5', learning_rate: 'Learning rate, default is 0.01' = '0.01')
    :param training_data: Training data organized in the torchvision format/structure
    :param max_epochs: Maximum number of epochs for the training, default is 5
    :param learning_rate: Learning rate, default is 0.01



In [5]:
train = train_module_func(
    training_data=train_data, 
    max_epochs=5, 
    learning_rate=0.02)

score = score_module_func(
    model_input=train.outputs.model_output, 
    test_data=test_data)

eval = eval_module_func(scoring_result=score.outputs.score_output)

pipeline = Pipeline(nodes=[train, score, eval], outputs=eval.outputs, name='sample pipeline to run on kubeflow')

run = pipeline.submit_run(
    ws, 
    experiment_name='jietong', 
#    default_compute_target='cpu-cluster'
    default_compute_target='kubeflow-aks'
)

run.wait_for_completion()

TypeError: submit_run() got an unexpected keyword argument 'name'